# 对事件进行聚类
（只取训练集和测试集中出现的样本）

数据来源于Kaggle竞赛：Event Recommendation Engine Challenge，根据
    events they’ve responded to in the past
    user demographic information
    what events they’ve seen and clicked on in our app
预测用户对某个事件是否感兴趣

竞赛官网：
https://www.kaggle.com/c/event-recommendation-engine-challenge/data

由于事件众多，可以考虑对事件进行聚类
事件描述信息在events.csv文件：共110维特征
前9列：event_id, user_id, start_time, city, state, zip, country, lat, and lng.
event_id：id of the event, 
user_id：id of the user who created the event.  
city, state, zip, and country： more details about the location of the venue (if known).
lat and lng： floats（latitude and longitude coordinates of the venue）
start_time： 字符串，ISO-8601 UTC time，表示事件开始时间

后101列为词频：count_1, count_2, ..., count_100，count_other
count_N：事件描述出现第N个词的次数
count_other：除了最常用的100个词之外的其余词出现的次数

这里我们用count_1, count_2, ..., count_100，count_other属性做聚类，即事件用这些关键词来描述，可表示事件的类别

# 导入工具包

In [7]:
import numpy as np
import scipy.sparse as ss
import scipy.io as sio

#保存数据
import cPickle

事件太多（300w+），训练+测试集的事件没这么多，所有先去处理train和test，得到竞赛需要用到的事件和用户
然后对在训练集和测试集中出现过的事件和用户建立新的ID索引
先运行user_event.ipynb,
得到事件列表文件：PE_userIndex.pkl

# 读取之前算好的测试集和训练集中出现过的事件

In [2]:
#读取训练集和测试集中出现过的事件列表
eventIndex = cPickle.load(open("PE_eventIndex.pkl", 'rb'))
n_events = len(eventIndex)

print("number of events in train & test :%d" % n_events)

number of events in train & test :13418


# 读取events的count特征

In [8]:
#读取训练集和测试集中出现过的事件列表
eventContMatrix = sio.mmread(open("EV_eventPropMatrix.mtx", 'rb'))

# 根据count特征进行聚类

In [9]:
from sklearn.cluster import MiniBatchKMeans

# rank1的解决方案中有分别进行20、40、80、100聚类的结果作为新的特征
#相当于对count特征进行了特征约减
km = MiniBatchKMeans(n_clusters=100)
km.partial_fit(eventContMatrix)

#保存预测结果
event_clusters = km.predict(eventContMatrix)

cPickle.dump(event_clusters, open("EV_clusters.pkl", 'wb'))

MiniBatchKMeans(batch_size=100, compute_labels=True, init='k-means++',
        init_size=None, max_iter=100, max_no_improvement=10,
        n_clusters=100, n_init=3, random_state=None,
        reassignment_ratio=0.01, tol=0.0, verbose=0)